In [1]:
import pandas as pd
import os
import gzip
import json
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import requests
import datetime
from scipy import stats
from load_helper_CoronaWiki import *
import urllib.parse
import time 
COVID_RELATED_ARTICLES_PATH = "data/COVID_related_pages_project-1.csv"
DICT_COUNTRIES_OWN_LANG = {"Italy" : "it", "Russia": "ru", "China": "zh", "India": "hi", "Albania": "sq", 
"Bangladesh": "bn", "Bostwana": "tn", "Cambogia": "km", "Croatia": "hr", "Greece": "el", "Sweden": "sv", "Finland": "fi", "Norway": "no",
 "Malaysian": "ms", "Israel": "he", "Lithuania": "lt", "Serbia": "sr", "Slovakia": "sk", "Slovania": "sl", "Turkey": "tr", "Ukrain": "uk",
 "Vietnam": "vi"}

In [2]:
df_covid_articles = pd.read_csv("data/COVID_related_pages_project-1.csv")

In [4]:
df_covid_articles.loc[df_covid_articles.project == "sr.wikipedia"]

,page,project,url,wikilink
215,Tocilizumab,sr.wikipedia,https://sr.wikipedia.org/wiki/Tocilizumab,[[sr:Tocilizumab
392,Teški akutni respiratorni sindrom,sr.wikipedia,https://sr.wikipedia.org/wiki/Te%C5%A1ki_akutn...,[[sr:Teški akutni respiratorni sindrom
507,Коронавирусна болест,sr.wikipedia,https://sr.wikipedia.org/wiki/%D0%9A%D0%BE%D1%...,[[sr:Коронавирусна болест
1054,Пандемија вируса корона у Француској 2020.,sr.wikipedia,https://sr.wikipedia.org/wiki/%D0%9F%D0%B0%D0%...,[[sr:Пандемија вируса корона у Француској 2020.
1423,SARS-CoV-2,sr.wikipedia,https://sr.wikipedia.org/wiki/SARS-CoV-2,[[sr:SARS-CoV-2
1536,Пандемија вируса корона у Србији 2020.,sr.wikipedia,https://sr.wikipedia.org/wiki/%D0%9F%D0%B0%D0%...,[[sr:Пандемија вируса корона у Србији 2020.
2107,Категорија:Пандемија вируса корона 2019/20. по...,sr.wikipedia,https://sr.wikipedia.org/wiki/%D0%9A%D0%B0%D1%...,[[sr:Категорија:Пандемија вируса корона 2019/2...
2538,Категорија:Пандемија вируса корона 2019/20.,sr.wikipedia,https://sr.wikipedia.org/wiki/%D0%9A%D0%B0%D1%...,[[sr:Категорија:Пандемија вируса корона 2019/20.
3012,Karneval u Veneciji,sr.wikipedia,https://sr.wikipedia.org/wiki/Karneval_u_Veneciji,[[sr:Karneval u Veneciji
3051,Категорија:Умрли од болести COVID-19,sr.wikipedia,https://sr.wikipedia.org/wiki/%D0%9A%D0%B0%D1%...,[[sr:Категорија:Умрли од болести COVID-19


In [8]:
df2 = df_covid_articles.groupby(['project']).aggregate('count')
df2 = df2.loc[df2.page > 10]
df2.shape

(60, 3)

In [31]:
'''
Function to be used within data fetching script
Inputs : 
    - json_obj : dict that was fetched in the given iteration of the script
    - return_df : final df that will be return at the end of the script
Output : concatenated df
'''
def json_to_df(json_obj, return_df):
    #Create df from json with nested list
    df = pd.json_normalize(json_obj, record_path = ['items']).set_index(['timestamp']).drop(labels = ['project', 'granularity'
    , 'access', 'agent', 'article'], axis = 1)
    #concatenation
    return  pd.concat([return_df, df])


In [46]:

def wiki_to_df_extract(languageCode, begin_date, end_date):
    df_covid_articles = pd.read_csv(COVID_RELATED_ARTICLES_PATH)
    df_covid_articles_country = df_covid_articles.loc[df_covid_articles.project == "{}.wikipedia".format(languageCode)]['page']
    df_agg_country = pd.DataFrame()
    for page in df_covid_articles_country:
        #time.sleep(0.05)
        page = page.replace(' ', '_')
        page = urllib.parse.quote(page)
        url = 'https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/{}.wikipedia.org/all-access/user/{}/daily/{}/{}'.format(languageCode,page, begin_date, end_date)
        header = {'User-Agent' : 'Robin Debalme (academic project; robin.debalme@epfl.ch; https://github.com/epfl-ada/ada-2022-project-thedatadiggers22)'}
        try:
            r = requests.get(url, headers = header).json()
        except:
            print("Bad request")
        df_agg_country = json_to_df(r, df_agg_country)
    return df_agg_country.groupby(['timestamp'])['views'].sum().to_frame()

In [50]:
#dict = {"Italy" : "it", "Russia": "ru"}
df_covid_agg = pd.DataFrame()
df_tmp  = pd.DataFrame()
for value in DICT_COUNTRIES_OWN_LANG:
    try:
        df_tmp = wiki_to_df_extract(DICT_COUNTRIES_OWN_LANG[value], '20200101', '20210101').rename({'views': value}, axis='columns')
        df_covid_agg = pd.concat([df_covid_agg, df_tmp], axis= 1)
    except:
        print(value)
#df_test = 
print(df_covid_agg)


Cambogia
Croatia
Serbia
            Italy  Russia  China  India  Albania  Bangladesh  Bostwana  \
timestamp                                                                
2020010100    246     409   4145     13      0.0         NaN       NaN   
2020010200    413     425  10676     20      NaN         NaN       NaN   
2020010300    396     711   6177     15      0.0         NaN       NaN   
2020010400    759    1694   6562     23      0.0         NaN       NaN   
2020010500   1153    1764   7149     40      0.0         NaN       NaN   
...           ...     ...    ...    ...      ...         ...       ...   
2020122800  15120   17387  26010   2973    202.0       530.0       4.0   
2020122900  15786   17492  26225   1723    218.0       617.0       7.0   
2020123000  15231   16782  26971   1701    178.0       733.0       8.0   
2020123100  14620   14407  23899   1316    105.0       681.0       5.0   
2021010100  13686   14456  20113    930    172.0       576.0       5.0   

            G

In [59]:
file = open('data/aggregated_timeseries.json')
data = json.load(file)

In [61]:
print(data)

AttributeError: 'dict' object has no attribute 'head'